In [1]:
import urllib
import dateutil
import matplotlib.pyplot as plt
import pandas
import datetime
import time
import json
import twstock
import matplotlib.dates as mpl_dates
from pathlib import Path
import mplfinance as mpf


In [2]:
twstock.__update_codes()


In [3]:
tickers = twstock.twse
df_tickers = pandas.DataFrame(tickers).T


In [4]:
'1101' in tickers


True

In [14]:
def getStockInfo(stock_num):
    try:
        # 取得證交所股票data
        stock = twstock.Stock(stock_num)
        df = pandas.DataFrame({
            'date': stock.date,
            'open': stock.open,
            'high': stock.high,
            'low': stock.low,
            'close': stock.price,
            'volume': stock.capacity,  # 使用 twstock.Stock.capacity 來取得成交量
        }, index=pandas.to_datetime(stock.date, format='%Y%m%d'))
        return df
        ##return stock
    except:
        print("Error")
    finally:
        time.sleep(15)


In [ ]:
stock = getStockInfo('2330')
stock_price = stock.price
stock_name = stock.sid
date = stock.date  # 時間
open = stock.open  # 開
high = stock.high  # 高
ow = stock.low  # 低
close = stock.price  # 收
date_format = mpl_dates.DateFormatter("%M %d")

print(stock)
print(stock_name)


In [21]:
### 日K線
stock = getStockInfo('1101')

mc = mpf.make_marketcolors(up='r', down='g', inherit=True)
s = mpf.make_mpf_style(base_mpf_style='yahoo', marketcolors=mc)
# 針對線圖的外觀微調，將上漲設定為紅色，下跌設定為綠色，符合台股表示習慣
# 接著把自訂的marketcolors放到自訂的style中，而這個改動是基於預設的yahoo外觀
kwargs = dict(type='candle', mav=(5, 20, 60), volume=True,savefig = dict(fname='testK.jpg'),
              figratio=(10, 8), figscale=0.75, title=stock_name, style=s)
# 設定可變參數kwargs，並在變數中填上繪圖時會用到的設定值
mpf.plot(stock, **kwargs)


In [ ]:
import requests
from pathlib import Path


def SendLineNotify(msgString):
    lineToken = 'HcmWNJxxh6s07aL65t05LCzNudclQasRz8VoacOBRuf'  # 個人測試用
    url = "https://notify-api.line.me/api/notify"
    payload = {'message': {msgString}}
    image = Path('./test.png').read_bytes()
    imageFile = {'imageFile': image}

    headers = {'Authorization': 'Bearer ' + lineToken}
    response = requests.request(
        "POST", url, headers=headers, data=payload, files=imageFile)
    print(response.text)


SendLineNotify("test")


In [ ]:

# 刪除檔案路徑
file = Path("/test.png")
file.unlink()


In [ ]:
# 取得即時資料
twstock.realtime.mock = False
stockRealTime = twstock.realtime.get('2330')
print(stockRealTime['success'])
stockRealTime


In [ ]:

# 取得多檔即時資料
stocks = twstock.realtime.get(['1101', '2409', '2301', '3019'])
df_realtime = pandas.DataFrame(stocks)
stocks
df_realtime


In [ ]:
from twstock import Stock
from twstock import BestFourPoint

stock = Stock('8454')
bfp = BestFourPoint(stock)

bfp.best_four_point_to_buy()    # 判斷是否為四大買點
# bfp.best_four_point_to_sell()   # 判斷是否為四大賣點
bfp.best_four_point()


In [ ]:
def get_two_float(f_str, n):
    a, c = f_str.partition('.')
    c = c[:n]
    return ".".join([a, c])
# 把回傳數字保留小數後兩位的function


In [ ]:
test = '10.11'
ans = int(float(test))


In [ ]:
import schedule


def test():
    print(time)


schedule.every(1).minutes.do(test)
while True:
    schedule.run_pending()
    time.sleep(1)


In [ ]:
import urllib
import dateutil
import matplotlib.pyplot as plt
import pandas
import datetime
import time
import json
import twstock
import requests
import schedule


def getFormateFloat(NumberString):
    if NumberString != "-":
        NumberString = int(float(NumberString))
        return NumberString


def SendLineNotify(msgString):
    lineToken = 'HcmWNJxxh6s07aL65t05LCzNudclQasRz8VoacOBRuf'  # 個人測試用
    url = "https://notify-api.line.me/api/notify"
    payload = {'message': {msgString}}
    headers = {'Authorization': 'Bearer ' + lineToken}
    response = requests.request("POST", url, headers=headers, data=payload)
    print(response.text)


def GetStockInfo():
    # 匯入
    companyList = ['2330', '2201']

    # 取得多檔即時資料
    stocks = twstock.realtime.get(companyList)
    df_realtime = pandas.DataFrame(stocks)

    for item in companyList:
        currentPrice = getFormateFloat(
            stocks[item]["realtime"]["latest_trade_price"])
        openPrice = getFormateFloat(stocks[item]["realtime"]["open"])
        lowPrice = getFormateFloat(stocks[item]["realtime"]["low"])
        highPrice = getFormateFloat(stocks[item]["realtime"]["high"])
        estimatePrice = getFormateFloat(openPrice * 0.03)
        tradeVolume = stocks[item]["realtime"]["trade_volume"]

        msg = (f' \n {stocks[item]["info"]["name"]} {stocks[item]["info"]["code"]} \n low:{lowPrice} ||high:{highPrice}  \n last_price {currentPrice}||成交量 {tradeVolume} \n 開盤價 {openPrice}')
        # 當有最新成交價才寄出
        if (currentPrice != "-"):
            SendLineNotify(msg)


schedule.every(1).minutes.do(GetStockInfo)
while True:
    schedule.run_pending()
    time.sleep(1)


In [ ]:
import pyautogui
import requests
from PIL import Image
import io

myScreenshot = pyautogui.screenshot()   # 截圖
myScreenshot.save('./test.png')         # 儲存為 test.png

url = 'https://notify-api.line.me/api/notify'
token = '你的權杖'
headers = {
    'Authorization': 'Bearer ' + token    # 設定 LINE Notify 權杖
}
data = {
    'message': '測試一下！'      # 設定 LINE Notify message ( 不可少 )
}
with open('./test.png', 'rb') as f:
    image = f   # 以二進位方式開啟圖片
imageFile = {'imageFile': image}   # 設定圖片資訊
data = requests.post(url, headers=headers, data=data,
                     files=imageFile)   # 發送 LINE Notify


In [ ]:
# open(r"./hello.txt","r") as file1:
from pathlib import Path
data = Path('./test.png').read_bytes()
print(data)


In [ ]:
# 無聊 預測奇偶數機率
import random
import pandas as pd


def predict_next_odd_even(training_data):
    odd_count = sum(1 for num in training_data if num % 2 == 1)
    even_count = len(training_data) - odd_count

    if odd_count > even_count:
        prediction = "奇數"
    elif even_count > odd_count:
        prediction = "偶數"
    else:
        prediction = random.choice(["奇數", "偶數"])

    return prediction


def calculate_number_probabilities(training_data):
    odd_numbers = [num for num in training_data if num % 2 == 1]
    even_numbers = [num for num in training_data if num % 2 == 0]

    odd_probability = format(
        len(odd_numbers) / len(training_data) * 100, '.2f')
    even_probability = format(
        len(even_numbers) / len(training_data) * 100, '.2f')

    probabilities = {
        "奇數": odd_probability,
        "偶數": even_probability
    }

    return probabilities


def display_probability_table(probabilities):
    df = pd.DataFrame(probabilities.items(), columns=['類型', '機率'])
    print(df)


training_data = []
for i in range(6):
    number = int(input(f"請輸入第 {i+1} 個數字作為訓練資料："))
    training_data.append(number)

next_prediction = predict_next_odd_even(training_data)
print("預測的下一組號碼類型是：", next_prediction)

probabilities = calculate_number_probabilities(training_data)
print("\n下一組號碼類型的機率表格：")
display_probability_table(probabilities)


In [ ]:
# 無聊幸運數字遊戲
import random
import pandas as pd

random_number_list = []

# 抽取隨機號碼


def take_random_number():
    df_num = random.randrange(1, 29)
    return df_num

# 隨機號碼抽獎比對


def random_number_check(df_num: int, num: int):
    random_number_list.append(df_num)

    if num == df_num:
        print("恭喜猜中")
        win_bool = True
    else:
        print("可惜猜錯")
        win_bool = False
    # 匯出表格
    display_probability_table(random_number_list)
    return win_bool

# 計算金額


def proess_money(money_sum: int, money: int, win_bool: bool):
    # 猜中 投注金額*5
    if win_bool == True:
        money = money * 5
        money_sum *= money
    else:
        money_sum -= money

    print("目前您總投注金額為:", money_sum)
    return money_sum

# 匯出表格


def display_probability_table(probabilities):
    df = pd.DataFrame(probabilities, columns=['中獎號'])
    print(df)

# 檢核輸入來源


def check_number(num):
    resultBool = True

    if str.isdigit(num) == False:
        print("請輸入數值")
        resultBool = False
    else:
        if int(num) < 1 and int(num) > 29:
            print("請輸入大於1且小於30的值")
            resultBool = False

    return resultBool

# 主體
def game_main():
    game_round_num = int(input(f"請輸入要進行幾回合遊戲："))
    money_sum = int(input(f"請輸入總投注金額"))
    super_time = False
    for i in range(game_round_num):
        # 抽隨機號碼
        df_num = take_random_number()

        # 檢核總投注金額
        if money_sum <= 0:
            print("你已無總投注金額!!! 歡迎下次再來!!")
            return

        money = int(input(f"請輸入此次投注金額"))
        # 檢核投注金額
        if (money > money_sum):
            print("您投注金額超過總投注金額，下回合請注意!")
            return

        number = input(f"請輸入第 1 個數字：")
        print(f"此回合你選擇為:{1}", number)

        if check_number(number) == True:
            win_bool = random_number_check(df_num, number)

        money_sum = proess_money(money_sum, money, win_bool)


# 程序流程
print("即將開始遊戲!!!!!!!!!!!!!!!!!!!!")
game_main()


In [6]:
from typing import List

def longestCommonPrefix(strs: List[str]) -> str:
    if not strs:
        return ""

    if strs == [""]:
        return ""
        
    ansList = []
    ansString = ''
    ansInt =0
    min_len = min(len(s) for s in strs)  
    strA = min_len
    for strA in range(min_len):
        TestText = strs[0][strA]
        ansInt = 0
        for strB in strs:
           if (strB[strA:(strA+1)] == TestText):
              ansInt += 1
              if(ansInt >= len(strs)):
                  ansString += TestText
           else:    
              return ansString
           
    ansList.append(ansString)   
    return ansString 
    
str_list =  ["cir","car"]
result = longestCommonPrefix(str_list)
print("result",result)

result c


In [8]:
def isValid( s: str) -> bool:

    strArray = list(s)
    answer = False
    min_len = len(s)
    answer_num = 0
    first_num = strArray[0]
    print('min_len',min_len)
    last_one_num = strArray[(min_len-1)]
    print('last_one_num',last_one_num)
    last_num = 0
    num = 0
    i = 0
    if ((min_len % 2) != 0):
        return False
    
    while i < min_len:

        str_1 = strArray[i]
        match str_1:
            case "(":
                num =1
            case "[":
                num =2
            case "{" :
                num =3
            case ")":
                num = -1
            case "]":
                num = -2
            case "}":
                num = -3
        answer_num+= num

        if (answer_num < 0 and num < 0):
            print(False)
            return False

        if (num < 0 and last_num >0):
            if(last_num != abs(num)):
                return False
        
        last_num = num
                             
        i += 1

    if (answer_num == 0):
        answer = True

    return answer


str_list =  "[([]])"
result = isValid(str_list)
print("result",result)


min_len 6
last_one_num )
result True


In [4]:
##mergeTwoLists
def mergeTwoLists(list1:[],list2:[]):
    list_answer = []
    list_all = list1 + list2

    for x in list_all:
        list_answer.append(x)
    
    #for y in list2:
        #list_answer.append(y)
    
    list_answer.sort()
    return list_answer

answer = []
list1 = [1,2,4]
list2 = [1,3,4]
answer = mergeTwoLists(list1,list2)
print('answer',answer)

    

answer [1, 1, 2, 3, 4, 4]
